In [1]:
import tensorflow
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from tensorflow.keras.models import load_model
import json
import random
import tkinter
from tkinter import *

In [2]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\multi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\multi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
def cust_lem(word):
    AbuBakr = "Abu Bakr"
    return word

In [6]:
lemmatizer = WordNetLemmatizer()
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(cust_lem(w.lower())) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")


35 documents
7 classes ['Abu Bakr 1', 'Abu Bakr 2', 'General', 'Umar 1', 'Umar 2', 'juma', 'maghfirat']
50 unique lemmatized words ['1', '2', '6', 'abo', 'abu', 'allah', 'asbaab', 'aur', 'azeem', 'bakr', 'bhoolein', 'bhoolo', 'biography', 'chahiye', 'din', 'dua', 'dusra', 'first', 'foregiveness', 'hona', 'ijtemai', 'isteghfaar', 'istighfaar', 'juma', 'ka', 'karna', 'key', 'ki', 'kirdaar', 'lamha', 'life', 'maango', 'maghfirat', 'mein', 'na', 'of', 'part', 'pe', 'pehla', 'pucho', 'qabooliyat', 'rabb', 'rub', 'saat', 'second', 'series', 'sey', 'tadbeer', 'umar', 'ummat']
Training data created


C:\Users\multi\Anaconda3\envs\chatbot\lib\site-packages\ipykernel_launcher.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/200
35/35 [==============================] - 0s 5ms/step - loss: 1.9792 - accuracy: 0.1429
Epoch 2/200
35/35 [==============================] - 0s 437us/step - loss: 1.9069 - accuracy: 0.2286
Epoch 3/200
35/35 [==============================] - 0s 636us/step - loss: 1.8766 - accuracy: 0.2000
Epoch 4/200
35/35 [==============================] - 0s 660us/step - loss: 1.8032 - accuracy: 0.3714
Epoch 5/200
35/35 [==============================] - 0s 444us/step - loss: 1.6374 - accuracy: 0.4857
Epoch 6/200
35/35 [==============================] - 0s 518us/step - loss: 1.6047 - accuracy: 0.4857
Epoch 7/200
35/35 [==============================] - 0s 492us/step - loss: 1.6199 - accuracy: 0.4000
Epoch 8/200
35/35 [==============================] - 0s 658us/step - loss: 1.3351 - accuracy: 0.6286
Epoch 9/200
35/35 [==============================] - 0s 745us/step - loss: 1.3731 - accuracy: 0.5143
Epoch 10/200
35/35 [==============================] - 0s 480us/step - loss: 1.3241 - accuracy

35/35 [==============================] - 0s 519us/step - loss: 0.0539 - accuracy: 0.9714
Epoch 82/200
35/35 [==============================] - 0s 639us/step - loss: 0.0496 - accuracy: 1.0000
Epoch 83/200
35/35 [==============================] - 0s 464us/step - loss: 0.0634 - accuracy: 1.0000
Epoch 84/200
35/35 [==============================] - 0s 480us/step - loss: 0.0280 - accuracy: 1.0000
Epoch 85/200
35/35 [==============================] - 0s 514us/step - loss: 0.0383 - accuracy: 1.0000
Epoch 86/200
35/35 [==============================] - 0s 742us/step - loss: 0.0368 - accuracy: 1.0000
Epoch 87/200
35/35 [==============================] - 0s 661us/step - loss: 0.0605 - accuracy: 1.0000
Epoch 88/200
35/35 [==============================] - 0s 654us/step - loss: 0.1061 - accuracy: 0.9429
Epoch 89/200
35/35 [==============================] - 0s 608us/step - loss: 0.0675 - accuracy: 1.0000
Epoch 90/200
35/35 [==============================] - 0s 662us/step - loss: 0.0445 - accuracy: 

35/35 [==============================] - 0s 462us/step - loss: 0.0103 - accuracy: 1.0000
Epoch 161/200
35/35 [==============================] - 0s 357us/step - loss: 0.0048 - accuracy: 1.0000
Epoch 162/200
35/35 [==============================] - 0s 382us/step - loss: 0.0319 - accuracy: 1.0000
Epoch 163/200
35/35 [==============================] - 0s 405us/step - loss: 0.0199 - accuracy: 1.0000
Epoch 164/200
35/35 [==============================] - 0s 447us/step - loss: 0.0163 - accuracy: 1.0000
Epoch 165/200
35/35 [==============================] - 0s 448us/step - loss: 0.0139 - accuracy: 1.0000
Epoch 166/200
35/35 [==============================] - 0s 415us/step - loss: 0.0108 - accuracy: 1.0000
Epoch 167/200
35/35 [==============================] - 0s 462us/step - loss: 0.0168 - accuracy: 1.0000
Epoch 168/200
35/35 [==============================] - 0s 511us/step - loss: 0.0442 - accuracy: 1.0000
Epoch 169/200
35/35 [==============================] - 0s 455us/step - loss: 0.0209 - a

In [7]:

lemmatizer = WordNetLemmatizer()

model = load_model('model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res



#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()
